In [13]:
p_storage_dir = "../../storage"

In [14]:
import pandas as pd
import re
from pathlib import Path

DA_STORAGE_DIR = Path(f"{p_storage_dir}/dados_abertos")

## Unidades

In [15]:
unidades = pd.read_csv(DA_STORAGE_DIR / "transformed/unidades.csv", delimiter=";")

unidades.head()

,nome,sigla,endereco,site,telefone,email,cnpj,uasg,autoridade_maxima_unidade
0,CÂMPUS ÁGUAS LINDAS,LIN,"Rua 21, Área Especial 4, Jardim Querência, CEP...",http://www.ifg.edu.br/aguaslindas,(61) 3799-7500,gabinete.aguaslindas@ifg.edu.br,10.870.883/0014-69,158640.0,Tiago Gomes de Araujo
1,CÂMPUS ANÁPOLIS,ANA,"Avenida Pedro Ludovico, s/ nº, Reny Cury, CEP ...",http://www.ifg.edu.br/anapolis,(62) 3703-3350,gabinete.anapolis@ifg.edu.br,10.870.883/0009-00,158522.0,Katia Cilene Costa Fernandes
2,CÂMPUS APARECIDA DE GOIÂNIA,APA,Avenida Universitária Vereador Vagner da Silva...,http://www.ifg.edu.br/aparecida,(62) 3507-5950,gabinete.aparecida@ifg.edu.br,10.870.883/0010-35,158610.0,Eduardo de Carvalho Rezende
3,CÂMPUS CIDADE DE GOIÁS,GOI,"Rua 02, Quadra 10, Lote 01 a 15, Residencial B...",http://www.ifg.edu.br/goias,(62) 3371-9000,gabinete.goias@ifg.edu.br,10.870.883/0011-16,158611.0,Sandro Ramos de Lima
4,CÂMPUS FORMOSA,FOR,"Rua 64 esquina com Rua 11, S/Nº, Expansão Parq...",http://www.ifg.edu.br/formosa,(61) 3642 9450,gabinete.formosa@ifg.edu.br,10.870.883/0007-30,158523.0,Thiago Goncalves Dias


# Remoção de registros inconsistentes

Neste conjunto de dados existe o registro de uma unidade IFG que aparentemente é utilizado para representar a unidade raiz do IFG, servindo como um agrupador pai para todas as outras. 

No nosso caso ela não é necessária, pois o nosso grafo já é sobre o IFG, então essa unidade será excluída.

In [16]:
unidades.iloc[14].to_frame().T

,nome,sigla,endereco,site,telefone,email,cnpj,uasg,autoridade_maxima_unidade
14,IFG,IFG,", , CEP , -",http://www.ifg.edu.br/,NaN,ifg@ifg.edu.br,NaN,NaN,NaN


In [17]:
unidades.drop(index=14, inplace=True)

## Coluna Endereço

- Atomizar os dados de endereço, dispersando-os em colunas

### Tratamento dos números

Os números do endereço podem envolver quadra e lote, ou quilômetro da rodovia, logo um simples split não é suficiente

In [ ]:
unidades["endereco"][0]

In [ ]:
enderecos_raw = unidades.values[:, 2]
enderecos_cleaned = []

# Separa os campos em um array e faz o strip() em cada um
for endereco in enderecos_raw:
    row = []
    
    for field in endereco.split(","):
        row.append(field.strip())
        
    enderecos_cleaned.append(row)

# Armazena os números antigos para comparar o resultado do pré-processamento
numeros_before = []

for endereco in enderecos_cleaned:
    temp = " ".join(endereco[1:3]) if re.match(r"(quadra |lote |qd\.? |lt\.? )", " ".join(endereco[1:3]), flags=re.IGNORECASE) else endereco[1]
    numeros_before.append(temp)
                    
# Padroniza a nomenclatura da numeração
# Quadra  -> Qd.
# Lote    -> Lt.
# nº 64   -> 64
# km 89   -> Km 89
numeros = []

for endereco in enderecos_cleaned:
    sem_numero_regex = r"s\/\s*nº"
    num_ordinal_regex = r"nº\s*"
    quilometro_regex = r"km\s*"
    
    if re.match(r"(qd\.?\s|lt\.?\s)", " ".join(endereco[1:3]).lower(), flags=re.IGNORECASE):
        numero_composto = " ".join(endereco[1:3])
        numero_composto = re.sub(r"qd\.?", "Qd.", numero_composto, flags=re.IGNORECASE)
        numero_composto = re.sub(r"lt\.?", "Lt.", numero_composto, flags=re.IGNORECASE)
        numeros.append(numero_composto)
    elif "quadra" in endereco[1].lower() and "lote" in endereco[2].lower():
        numeros.append(" ".join(endereco[1:3]).lower().replace("quadra", "Qd.").replace("lote", "Lt."))
    elif "quadra" in endereco[1].lower():
        numeros.append(endereco[1].lower().replace("quadra", "Qd."))
    elif re.match(sem_numero_regex, endereco[1], re.IGNORECASE) != None:
        numeros.append(re.sub(sem_numero_regex, "S/N", endereco[1], flags=re.IGNORECASE))
    elif re.match(num_ordinal_regex, endereco[1], re.IGNORECASE):
        numeros.append(re.sub(num_ordinal_regex, "", endereco[1], flags=re.IGNORECASE))
    elif re.match(quilometro_regex, endereco[1], re.IGNORECASE):
        numeros.append(re.sub(quilometro_regex, "Km ", endereco[1], flags=re.IGNORECASE))
    else:
        numeros.append(endereco[1])
        
pd.DataFrame({"antes": numeros_before, "depois": numeros})

### Tratamento dos CEPs

Os CEPs dos endereços estão seguindo o padrão: "CEP XXXXX-XXX" mas para diminuir a redundância do grafo, queremos apenas o número do CEP

In [ ]:
ceps = []
ceps_before = []

cep_regex = r"CEP\s*"

for endereco in enderecos_cleaned:
    # Armazenar os antigos CEPs para comparar os resultados do pré-processamento
    ceps_before.append(endereco[-2])
    
    cep = re.sub(cep_regex, "", endereco[-2], flags=re.IGNORECASE)
    
    if cep.find("-") == -1:
        cep = "-".join([cep[:5], cep[-3:]])
    
    ceps.append(cep)
    
pd.DataFrame({"antes": ceps_before, "depois": ceps})

### Tratamento do nome da cidade

O nome das cidades estão com a UF no final, mas sabemos que os IFG do dataset estão todos localizados em Goiás, logo para os dados ficarem mais limpos é necessário remover a sigla da UF do nome da cidade.

In [ ]:
cidades = []

for endereco in enderecos_cleaned:
    cidades.append(endereco[-1][0:-5].upper())
    
pd.DataFrame(cidades, columns=["cidade"])

### Extração do logradouro e bairro do dataset inicial

In [ ]:
logradouros = []
bairros = []

for endereco in enderecos_cleaned:
    logradouros.append(endereco[0])
    bairros.append(endereco[-3])
    

pd.DataFrame({"logradouro": logradouros, "bairro": bairros})

### Dataframe com os endereços atomizados

In [ ]:
enderecos_preprocessed = pd.DataFrame({"logradouro": logradouros, "numero": numeros, "bairro": bairros, "cep": ceps, "cidade": cidades})

enderecos_preprocessed

### Dataframe final

In [ ]:
df_final = pd.DataFrame(unidades.iloc[:, :2]).join(enderecos_preprocessed).join(unidades.iloc[:, 3:-1])

df_final

In [ ]:
df_final.head()

In [ ]:
export_path = Path(DA_STORAGE_DIR / "preprocessed")

if not export_path.is_dir():
    Path(export_path).mkdir()

try:
    df_final.to_csv(export_path / "unidades.csv", sep=";", index=False)
except Exception as e:
    print(str(e))